1. Cargar el data set
1. Hacer el analisis de forma, cantidades de nulos, value_count de tipo de propiedad etc
1. Agrupar por provincia para que se vea que capital tiene mas datos
1. Value_counts de CABA
1. Desglosar la columna parent_name y tomar barrio (Palermo en vez de Palermo Soho, etc)
1. Limpiar los datos de barrios
1. Nuevos dataframes: Buenos Aires + el analisis de parent_name
1. Chequear tasa de cambio de dolares a pesos con las respectivas pruebas de que este ok calculo
1. Crear una variable/campo/columna nueva con el precio en dolares (calculado como precio en pesos/ tasa de cambio)
1. Chequear nulos con umbral +9 y dropear 
1. Chequear la moneda y dropear los casos extraños
1. Agrupar por barrio y .describe() sobre precio_mt2
1. Box plot y ver outliers o casos llamativos. Graficar los que tengan mayor desvío
1. TOMAR DECISION: de acuerdo al analisis como imputamos los valores faltantes de precio por mt2
    1. Aplicar la media, mediana?
    1. Aplicar el valor del vecino mas cercano en funcion de la cantidad de metros (agrupar por barrio y ordenar por cantidad de metros totales)
    1. Hacer qcut o cut por cantidad de metros, aplicar la media sobre los intervalos  a los deptos que entren en esa cantidad de metros (esto es posible???)
1. Describe de nuevo para ver como cambian los valores con imputacion
1. Analisis del campo descrition: para ambientes y amemities (pileta/piscina, SUM, laundry/lavadero, cochera/garage, Gimnasio/gym, seguridad)
1. Convertir en dummies los valores del campo descripcion y tipo de propiedad
1. Grafico de distribucion con geolocalizacion / cantidad de ventas por barrio segun tipo de propiedad



**Notas**:
- En este momento hay muchas celdas que tiran error porque los objetos tienen nombres distintos! 
- Existe una forma de parsear URLs [URL parser](https://docs.python.org/3/library/urllib.parse.html).<br>
Lo nombramos pero no profundizamos. No sé si tiene mucho sentido adentrarnos en ese analisis y vamos a usar el campo description 
- Traté de ordenar todo lo que ya estaba hecho como lo pusimos en la lista. Lo de la distribucion de MONEDA me quedo antes pero se puede pasar abajo sin problemas. Tambien todo lo que refiere al calculo de tasa de cambio y de precio_uss podria hacerse tanto antes de filtrat caba como despues.
- Una vez que hayamos completado la nb, hay que chequear consistenecia en nombres de variables, objetos, etc etc
- Tambien habria que dar una leida a todos los textos del markdown, dar consistencia a la sinxtaxis (markdown o html) y completar las explicaciones o comentarios del codigo que hagan falta
- Armar el table of contents del archivo (Jazmin)
- Todo lo que no supe como hacer y ya no me daba la cabeza lo deje comentado para retomar despues

<h1>Desarrollo del Desafio 1</h1>

<p>La inmobiliaria Properati publica periódicamente información sobre ofertas de propiedades para venta y alquiler. Ud. deberá asesorar a la inmobiliaria a desarrollar un modelo de regresión que permita predecir el precio por metro cuadrado de una propiedad. El objetivo             
final es que el modelo que desarrollen sea utilizado como tasador automático a ser aplicados a las próximas propiedades que sean comercializadas por la empresa. Para ello la empresa le provee de un dataset correspondiente al primer semestre de 2017.</p>

El dataset es de tamaño entre pequeño y mediano, pero tiene dos complejidades a las que deberá prestarle atención:
 
<ul>
    <li>Peso de missing data en algunas variables relevantes.</li>
    <li>Será importante tener en cuenta el problema de la influencia espacial en los precios por metro cuadrado. En efecto, es probable que existan diferencias importantes de en las diferentes geografías, barrios y zonas analizadas.</li>
</ul>

<h2>Objetivos:</h2>

<ul>
    <li>Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.</li>
    <li>Realizar un análisis descriptivo de las principales variables.</li>
    <li>Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.</li>
</ul>

### Importar las librerias que vamos a utilizar ###

In [126]:
import numpy as np
import pandas as pd
import chardet
import re

# graficos
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import NumeralTickFormatter # Para imprimir los %

### 1. Cargar el data set y verificar que los datos se hayan importado correctamente ###

In [127]:
# Funcion especial para autodetectar el encoding. La eliminamos porque demora mucho!!!
# def get_encoding_type(csv_path):
#     rawdata = open(csv_path, 'rb').read()
#     result = chardet.detect(rawdata)
#     return result.get('encoding')

In [128]:
#Se define la ruta de la información.
data_propiedades = "../Data/properati.csv"
#Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")
#Chequeamos que los datos se hayan importado correctamente
data.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [129]:
data.tail()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121216,121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,...,360.0,1383.333333,1383.333333,NaN,NaN,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...
121218,121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,...,48.0,1997.916667,1997.916667,NaN,NaN,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...
121219,121219,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,77.0,1675.324675,1675.324675,NaN,NaN,3500.0,http://www.properati.com.ar/1cjaa_venta_depart...,Departamento de 77 m2 apto profesional. Edific...,Departamento apto profesional en el Centro,https://thumbs4.properati.com/8/mNEsnQWqB7joBF...


### 2. Hacer el analisis de forma, cantidad de registros, cantidades de nulos por campo, value_counts ###

In [130]:
#Chequeamos cantidad de registros y cantidad de variables
data.shape

(121220, 26)

In [131]:
#Vemos qué tipos de datos tienen los campos
data.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

In [132]:
#renombramos la columba "Unnamed 0"
data.rename(columns={'Unnamed: 0': "Index"}, inplace=True)
data.head(3)

,Index,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


#### Calculos de cantidad de nulos ####

In [133]:
#Calculamos cantidad de nulos por campo
cant_nulos_por_campo = data.isnull().sum()
cant_nulos_por_campo

Index                              0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

In [134]:
#Calculamos porcentaje de nulos por campo
cant_registros = data.shape[0]
print(f'Cantidad Original de registros: {cant_registros}\n')
#len(data.index) es lo mismo

porc_nulos_por_campo = np.round((100 * cant_nulos_por_campo / cant_registros),2)
print(porc_nulos_por_campo)

Cantidad Original de registros: 121220

Index                          0.00
operation                      0.00
property_type                  0.00
place_name                     0.02
place_with_parent_names        0.00
country_name                   0.00
state_name                     0.00
geonames_id                   15.44
lat-lon                       42.53
lat                           42.53
lon                           42.53
price                         16.84
currency                      16.84
price_aprox_local_currency    16.84
price_aprox_usd               16.84
surface_total_in_m2           32.44
surface_covered_in_m2         16.42
price_usd_per_m2              43.39
price_per_m2                  27.69
floor                         93.48
rooms                         60.91
expenses                      88.23
properati_url                  0.00
description                    0.00
title                          0.00
image_thumbnail                2.57
dtype: float64


#### Observamos la distribución de distintos campos que pueden aportar al analisis ####

In [135]:
#Observamos distribucion de elementos en campo property_type y en el campo operation
property_type_count = data.property_type.value_counts()
print(f'property_type: \n{property_type_count[0]}')

property_operation_count = data.operation.value_counts()
print(f'operation: \n{property_operation_count[0]}')

property_type: 
71065
operation: 
121220


In [136]:
categories = data.property_type.unique()
cord_x = data.property_type.value_counts() / data.shape[0]

palette = ["#99d594", "#1ad594", "#feffbf", "#f33def"]

p = figure(x_range=categories, plot_width=500, plot_height=200)
p.vbar(x=categories, top=cord_x, width=0.6,
       color=palette)

p.yaxis.formatter = NumeralTickFormatter(format='0 %')

output_notebook()

show(p)

Loading BokehJS ...

In [137]:
#Observamos que todos los registros corresponden a Argentina
country_name_count = data.country_name.value_counts()
print(f'Country: {country_name_count[0]}')

#Todos los valores de la columna country_name son Argentina 
#de la columna place_with_parents_name se puede reemplazar Argentina por string vacio

len(data.place_with_parent_names.index)
# data.place_with_parent_names.sample(30)

Country: 121220


121220

### Distribucion del tipo de moneda ###

In [138]:
data.currency.value_counts()

USD    87587
ARS    13219
PEN        2
UYU        1
Name: currency, dtype: int64

In [139]:
# Creo una mask para traer esos valores.
currency_OTHER_CURRENCY_mask = (data.currency == 'PEN') | (data.currency == 'UYU')
data.loc[currency_OTHER_CURRENCY_mask, :]

,Index,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
50387,50387,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e8z_venta_depart...,Ref.: 55:PREVENTA ::: OPORTUNIDAD ::: Dúplex a...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/9/N_Qr_5X_vOq--b...
50388,50388,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e90_venta_depart...,Ref.: 56Duplex en venta de 1 dormitorio con ap...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/8/_cBXAyr_9pDySg...
107390,107390,sell,house,San Roque,|Argentina|Mendoza|San Roque|,Argentina,Mendoza,3836586.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1by0s_venta_casa_s...,"Casa con Calefacción central, Mendoza, San Roq...",Casa - San Vicente,https://thumbs4.properati.com/2/pA0RbuDKRK6GgV...


In [140]:
# SE PUEDE LIMPIAR - SIMPLIFICADO CON EL COMANDO DE ARRIBA.
# Creo una mask para traer esos valores.
# currency_UYU_mask = data.currency == 'UYU'
# data.loc[currency_UYU_mask, :]

#### Visto que no se puede garantizar que estos datos estén cargados correctamente y son solo tres registros en todo el data set, se toma la decision de eliminarlos ####
Chequear que este codigo para dropear esté correcto! 

In [141]:
# data.drop(data.loc[currency_PEN_mask].index, inplace = True)
# data.drop(data.loc[currency_UYU_mask].index, inplace = True)
data.drop(data.loc[currency_OTHER_CURRENCY_mask, :].index, inplace = True)

print(f'Cantindad de registros original - los eliminados: {cant_registros - data.shape[0]}')
print('\nEsta Chequedado!')

Cantindad de registros original - los eliminados: 3

Esta Chequedado!


### 3. Distribucion por provincia ###
**Nota**: aqui modifico lo que hizo Ignacio porque no creo todavia un dataframe nuevo

In [142]:
data.state_name.value_counts()

Capital Federal                 32314
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           680
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
Santiago Del

<h2>Les parece armar una distribución por CABA, BS.AS, y resto del pais? y ver los %</h2>

In [143]:
# data_precio.place_name.value_counts()

### calcular cantidad de nulos por campo por grupo? ###

### 4.5.6. Limpieza de datos de los barrios de caba y counts ###
Haremos un analisis de Capital Federal unicamente, ya que es un grupo que se distingue del resto por la cantidad y tipo de datos (en **place_with_parent_name**)

#### Generamos un nuevo data frame solo con los datos de CABA ####

In [144]:
data_caba_mask = data.state_name == 'Capital Federal'

data_caba = data.loc[data_caba_mask, :]

data_caba

,Index,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121160,121160,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.563685346,-58.4426825896",-34.563685,...,NaN,2611.464968,NaN,NaN,NaN,6300.0,http://www.properati.com.ar/1cj5v_venta_depart...,Edificio en esquina de muy buena categoría de ...,Av .Luis Maria Campos 1400- Belgrano-Capital F...,https://thumbs4.properati.com/0/zeLNBuyylOJL7z...
121209,121209,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.563685346,-58.4426825896",-34.563685,...,NaN,2611.464968,NaN,NaN,NaN,6300.0,http://www.properati.com.ar/1cj7c_venta_depart...,Edificio en esquina de muy buena categoría de ...,Av .Luis Maria Campos 1400- Belgrano-Capital F...,https://thumbs4.properati.com/0/zeLNBuyylOJL7z...
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...


Reseteamos el index del nuevo data_frame

In [145]:
data_caba.reset_index()


,index,Index,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
2,3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
3,7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",...,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
4,8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",...,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32309,121160,121160,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.563685346,-58.4426825896",...,NaN,2611.464968,NaN,NaN,NaN,6300.0,http://www.properati.com.ar/1cj5v_venta_depart...,Edificio en esquina de muy buena categoría de ...,Av .Luis Maria Campos 1400- Belgrano-Capital F...,https://thumbs4.properati.com/0/zeLNBuyylOJL7z...
32310,121209,121209,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.563685346,-58.4426825896",...,NaN,2611.464968,NaN,NaN,NaN,6300.0,http://www.properati.com.ar/1cj7c_venta_depart...,Edificio en esquina de muy buena categoría de ...,Av .Luis Maria Campos 1400- Belgrano-Capital F...,https://thumbs4.properati.com/0/zeLNBuyylOJL7z...
32311,121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
32312,121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...


#### Vemos la distribucion por barrio ####

In [146]:
data_caba.place_name.value_counts()

Belgrano           2992
Palermo            2885
Caballito          2273
Villa Urquiza      1629
Recoleta           1547
                   ... 
Velez Sarsfield      39
Villa Real           37
Villa Soldati        15
Villa Riachuelo       6
Catalinas             4
Name: place_name, Length: 62, dtype: int64

In [147]:
# print(precios_caba.count() / precios_caba.shape[0]) 
print('% de valores incompletos por columna:')
print(f'{round(data_caba.isnull().sum()/data_caba.count(),2)}')
print(' Cantidad de valores incompletos por columna:')
print(data_caba.isnull().sum())

% de valores incompletos por columna:
Index                         0.00
operation                     0.00
property_type                 0.00
place_name                    0.00
place_with_parent_names       0.00
country_name                  0.00
state_name                    0.00
geonames_id                   0.04
lat-lon                       0.36
lat                           0.36
lon                           0.36
price                         0.11
currency                      0.11
price_aprox_local_currency    0.11
price_aprox_usd               0.11
surface_total_in_m2           0.22
surface_covered_in_m2         0.08
price_usd_per_m2              0.38
price_per_m2                  0.18
floor                         9.74
rooms                         1.11
expenses                      3.90
properati_url                 0.00
description                   0.00
title                         0.00
image_thumbnail               0.02
dtype: float64
 Cantidad de valores incompletos por 

#### Limpiar el campo **place_with_parent_name** ####

1. Eliminar "Argentina" y "|" de los strings

In [148]:
#Creo una serie sobre la columna para trabajarla
serie_place_with_parent_names = data_caba.place_with_parent_names
serie_place_with_parent_names

0             |Argentina|Capital Federal|Mataderos|
2             |Argentina|Capital Federal|Mataderos|
3               |Argentina|Capital Federal|Liniers|
7              |Argentina|Capital Federal|Belgrano|
8              |Argentina|Capital Federal|Belgrano|
                            ...                    
121160         |Argentina|Capital Federal|Belgrano|
121209         |Argentina|Capital Federal|Belgrano|
121215         |Argentina|Capital Federal|Belgrano|
121217    |Argentina|Capital Federal|Villa Urquiza|
121219                  |Argentina|Capital Federal|
Name: place_with_parent_names, Length: 32314, dtype: object

In [149]:
#Primero sacamos el string el pais
pattern_arg = '^\|Argentina\|'
#RE busca |Argentina al comienzo del string
pattern_arg_regex = re.compile(pattern_arg)
#creo una variable nueva para hacer el apply y reemplazo "|Argentina" por string vacio 
place_parent_name_no_arg = serie_place_with_parent_names.apply(lambda x: pattern_arg_regex.sub("", x))

#chequeo los valores que quedaron
place_parent_name_no_arg.head()

0    Capital Federal|Mataderos|
2    Capital Federal|Mataderos|
3      Capital Federal|Liniers|
7     Capital Federal|Belgrano|
8     Capital Federal|Belgrano|
Name: place_with_parent_names, dtype: object

In [150]:
#Elimino los pipes innecesarios
pattern_pipe_final = '\|$'
#RE busca | al final del string
regex_pipe_final = re.compile(pattern_pipe_final)
#Reemplazo el pipe por string vacio así despues puedo splitear la cadena
place_parent_final = place_parent_name_no_arg.apply(lambda x: regex_pipe_final.sub("", x))
place_parent_final

0             Capital Federal|Mataderos
2             Capital Federal|Mataderos
3               Capital Federal|Liniers
7              Capital Federal|Belgrano
8              Capital Federal|Belgrano
                      ...              
121160         Capital Federal|Belgrano
121209         Capital Federal|Belgrano
121215         Capital Federal|Belgrano
121217    Capital Federal|Villa Urquiza
121219                  Capital Federal
Name: place_with_parent_names, Length: 32314, dtype: object

2. Splitear la cadena para obtener una lista

In [151]:
serie_list_place_split = place_parent_final.apply(lambda x: x.split("|"))
serie_list_place_split.head(10)
# type(serie_list_place_split)

0                 [Capital Federal, Mataderos]
2                 [Capital Federal, Mataderos]
3                   [Capital Federal, Liniers]
7                  [Capital Federal, Belgrano]
8                  [Capital Federal, Belgrano]
13    [Capital Federal, Palermo, Palermo Soho]
14    [Capital Federal, Palermo, Palermo Soho]
16                [Capital Federal, Mataderos]
19                  [Capital Federal, Palermo]
21                  [Capital Federal, Palermo]
Name: place_with_parent_names, dtype: object

In [152]:
#Evaluo las longitudes de las listas que me quedaron en la serie
serie_len = serie_list_place_split.apply(len)
serie_len.value_counts()
# serie_len.values

2    29821
1     1295
3     1198
Name: place_with_parent_names, dtype: int64

Vemos que hay valores en esa serie que tienen un solo elemento, los miramos para ver si sirven al análisis

In [153]:
#hacemos una mascara e indexamos la lista a traves de una lamba que compara la longitud de cada elemento
mask_len_1 = serie_list_place_split.apply(lambda x: len(x) == 1)
serie_list_place_split.loc[mask_len_1]

108       [Capital Federal]
110       [Capital Federal]
115       [Capital Federal]
119       [Capital Federal]
121       [Capital Federal]
                ...        
121098    [Capital Federal]
121102    [Capital Federal]
121124    [Capital Federal]
121125    [Capital Federal]
121219    [Capital Federal]
Name: place_with_parent_names, Length: 1295, dtype: object

In [154]:
# indexamos la lista para que nos devuelva los valores con 1 elementos distinto a 'Capital Federal'
mask_len_1_not_caba = serie_list_place_split.apply(lambda x: (len(x) == 1) & (x[0] != 'Capital Federal'))
serie_list_place_split[mask_len_1_not_caba]

Series([], Name: place_with_parent_names, dtype: object)

Observamos que no todos estos elementos contienen la especificidad del **Barrio** que es necesaria para hacer los calculos de mt2 por barrio 
### dropeamos esos registros ###

In [155]:
# place_clean = serie_list_place_split.drop
# place_clean


#RECORDAR CAMBIAR LOS OBJETOS EN LOS QUE SIGUE

3. Obtengo dos series de barrio y sub_barrio

In [156]:
#funciones para extraer el elemento que me interesa de cada lista según la posicion
#también considero la opcion de Capital Federal, donde la logica es distinta

def get_sub_barrio(x):
    return x[2] if len(x) == 3 else np.NaN
        
def get_barrio(x):
    return x[1] if (len(x) == 3 or len(x) == 2) else np.NaN

# def get_provincia(x):
#     return np.NaN if len(x) == 1 else x[0]

In [157]:
serie_sub_barrio = serie_list_place_split.apply(get_sub_barrio)

serie_barrio = serie_list_place_split.apply(get_barrio)

# serie_provincia = serie_lista_place_split.apply(get_provincia)

In [158]:
barrio_count = serie_barrio.value_counts()
barrio_count.sort_index()

Abasto                   113
Agronomía                 41
Almagro                 1165
Balvanera                667
Barracas                 425
Barrio Norte            1140
Belgrano                2992
Boca                     223
Boedo                    890
Caballito               2273
Catalinas                  4
Centro / Microcentro     223
Chacarita                227
Coghlan                  259
Colegiales               352
Congreso                 304
Constitución             211
Flores                  1354
Floresta                 516
Las Cañitas              143
Liniers                  408
Mataderos                418
Monserrat                538
Monte Castro              79
Nuñez                    778
Once                     198
Palermo                 4083
Parque Avellaneda         56
Parque Centenario         68
Parque Chacabuco         146
Parque Chas               44
Parque Patricios         322
Paternal                 173
Pompeya                   64
Puerto Madero 

### 7. Armamos un nuevo DF con CABA y los datos de los barrios sacando las columnas que no necesitamos ###

In [159]:
#armo un df nuevo para ver cómo quedó la separación
frame = {'Barrio': serie_barrio, 'Sub_Barrio': serie_sub_barrio}  
caba_place_names = pd.DataFrame(frame) 

data_caba_barrios = pd.concat([data_caba, caba_place_names], axis=1, sort=False)
data_caba_barrios.head(3)

#falta hacer filtro de columnas que no sirven

,Index,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,Barrio,Sub_Barrio
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,Mataderos,NaN
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,Mataderos,NaN
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,Liniers,NaN


### 8. Chequear tasa de cambio de dolares a pesos con las respectivas pruebas de que este ok calculo ###
### 9. Creacion de columna de precio en dolares ### 

In [160]:
data_caba_barrios['usdtolocal'] = data_caba_barrios.price_aprox_local_currency / data_caba_barrios.price_aprox_usd
cambio_usd_max = data_caba_barrios.usdtolocal.max()
cambio_usd_min = data_caba_barrios.usdtolocal.min()

print(data_caba_barrios.sample(3))

# Verifico si en el calculo el valor max y min son iguales.
print(round(cambio_usd_max, 2))
print(round(cambio_usd_min, 2))

check_change = round(data_caba.usdtolocal, 2) == round(cambio_usd_max, 2)

print(f'Son todos igual a {round(cambio_usd_max, 2)}: {all(check_change)}')

print(f'Hay valores que son NaN?: {any(data_caba_barrios.usdtolocal.isnull())}')

       Index operation property_type     place_name  \
26330  26330      sell     apartment  San Cristobal   
93397  93397      sell     apartment   Villa Crespo   
71624  71624      sell     apartment      Caballito   

                         place_with_parent_names country_name  \
26330  |Argentina|Capital Federal|San Cristobal|    Argentina   
93397   |Argentina|Capital Federal|Villa Crespo|    Argentina   
71624      |Argentina|Capital Federal|Caballito|    Argentina   

            state_name  geonames_id                       lat-lon        lat  \
26330  Capital Federal    3429153.0                           NaN        NaN   
93397  Capital Federal    3427458.0   -34.597985113,-58.426512964 -34.597985   
71624  Capital Federal    3435874.0  -34.6060340291,-58.452885322 -34.606034   

       ...  floor  rooms expenses  \
26330  ...    NaN    2.0      NaN   
93397  ...    NaN    NaN      NaN   
71624  ...    NaN    NaN      NaN   

                                           prope

AttributeError: 'DataFrame' object has no attribute 'usdtolocal'

Revisando tenemos Price y Price_aprox_usd que parecen ser lo mismo ¿? vamos a tratar de compararlos.

In [ ]:
# revisar price contra price_aprox_usd
check_prices_usd = data_caba_barrios.price == data_caba_barrios.price_aprox_usd
print(f'El valor de price es igual a price_aprox_usd?: {all(check_prices_usd)}')
print(f'Hay precios con campos NaN? : {any(data_caba_barrios.price.isnull())}') # Hay precios que son Nulos.

Con esto detectamos que el precio y precio aproximado parecen no ser lo mismo tampoco.

In [ ]:
#creamos una columna con el valor en dolares en funcion de price
data_caba_barrios['price_usd']= round(data_caba_barrios.apply(lambda x: x['price']  if x['currency'] == 'USD' else (x['price'] * x['usdtolocal'] if x['currency'] == 'ARS' else np.NaN), axis=1),2)
data_caba_barrios.sample(10)

Se puede observar que falta completar el **price_per_m2** en la moneda local. Pero!! se puede calcular en dolares la superficie cubierta porque tenemos el precio en USD por m2, **dividiendo el valor por el precio por metro cuadrado, obtendriamos el dato surface_covered_in_m2**

### 10. Chequeamos cantidad de nulos y dropeamos segun el umbral ###

In [ ]:
# print(precios_caba.count() / precios_caba.shape[0]) 
print('% de valores incompletos por columna:')
print(f'{round(data_caba_barrios.isnull().sum()/data_caba_barrios.count(),2)}')
print(' de valores incompletos por columna:')
print(data_caba_barrios.isnull().sum())

In [ ]:
# De los registros de CABA a cuantos les falta más de 10 datos por fila

precios_caba_mas10_mask = data_caba_barrios.isnull().sum(axis=1) > 9 

print(data_caba_barrios.loc[precios_caba_mas10_mask, :])

# Se puede observar que de estos datos no tenemos tanta información como para incluir y los podemos eliminar del set de datos!!
# Existe Palermo Chico ??  no tiene geonames_id ??

# En el caso de optar por eliminar podemos usar: 
# umbral = 9
# precios_caba_clean1 = data_caba.dropna(axis = 0, thresh=umbral)

<strong>Verifico Palermo Chico lo elimiaría si no puedo calcular el precio por metro cuadrado!! Ya que no aportan mucha información!</strong>
<strong>Se puede mejorar con algun grafico?<strong>

In [ ]:
data_caba_geo_mask = data_caba_barrios.geonames_id.isnull()

data_caba_geo = data_caba_barrios.loc[data_caba_geo_mask, :]

data_caba_geo.head()

# Parece que se puede completar sin problemas, aunque no veo el sentido. 

In [ ]:
precios_caba_place_mask = data_caba_barrios.place_name == 'Las Cañitas'
precios_caba_las_cañitas = data_caba_barrios.loc[precios_caba_place_mask, :]

precios_caba_las_cañitas.geonames_id.isnull().sum() / precios_caba_las_cañitas.shape[0]

# Lo que me dice que esta locación no tiene geonames_id !! revisar si se puede analizar automaticamente para el resto.
# Las Cañitas es la denominación informal y no oficial de un sector de unas 20 manzanas dentro del barrio de Palermo en la Ciudad Autónoma de Buenos Aires.

### 11. Tipo de moneda. Quedo como parte del punto 2 #

### 12. Agrupar por barrio y describe() por mt2 ### 

In [ ]:
barrio_group = data_caba_barrios.groupby('Barrio')

In [ ]:
barrio_group.price_per_m2.describe()

### 13. Box plot para ver outliers de los barrios y eliminar o ajustar antes de imputar datos ###

In [ ]:
#Ver cuales outliers de cada barrio y analizar el desvio ANTES de impurtar los mt2 promedio por barrio

### 14. Imputacion de datos faltantes ### Tomar una decision

In [ ]:
# Analicemos este unico registro. 
data_caba_barrios.loc[44469]

# Encontramos que no tiene el precio_usd_per_m2, y se puede calcular...de dos maneras:

# 1. price_usd / surface_covered_in_m2

# Al parecer cuando es un departamente no tiene surface_total_in_m2 !!

In [ ]:
# Calculo el valor del metro cuadrado para esta propiedad...

round(data_caba_barrios.loc[44469].price_usd / data_caba_barrios.loc[44469].surface_covered_in_m2, 2)

# Me da el mismo valor de price_per_m2 que debería estar el pesos! no en USD.

In [ ]:
# Contar todos los elementos de CABA que no tiene price_usd_per_m2

price_usd_per_m2_mask = data_caba_barrios.price_usd_per_m2.isnull()
print(price_usd_per_m2_mask.loc[44469]) # Verifico que la funcion de True para el valor que tengo.
price_usd_per_m2_isnan = data_caba_barrios.loc[price_usd_per_m2_mask, :]

# print(price_usd_per_m2_mask)
print(price_usd_per_m2_mask.value_counts()) # Me deberia dar 0 
print(f'{round(price_usd_per_m2_mask.sum() / data_caba_barrios.shape[0], 2)} % de valores con NaN')

# De esto ultimo se desprende que tenemos 27% de datos donde falta el valor de precio_usd_per_m2

price_usd_per_m2_isnan.sample(10)

Entonces ahora trato de reemplazar los valores faltantes operando con los datos que tengo.

In [ ]:
data_caba_barrios.price_usd_per_m2 = data_caba_barrios.price_usd_per_m2.replace(np.NaN, data_caba_barrios.price_usd / data_caba_barrios.surface_covered_in_m2)

# Investigar el error de abajo, y este proceso demora mucho tiempo.

#### Calcular precio promedio por barrio ####

In [ ]:
# precios_caba.price_per_m2 = precios_caba.price_per_m2.replace(precios_caba.groupby('place_name').price_per_m2.transform(lambda x: x.fillna(x.mean())))
# print(precios_caba.price_per_m2.replace(precios_caba.groupby('place_name').price_per_m2.transform(lambda x: x.fillna(x.mean()))))

# precios_caba.price_per_m2 = precios_caba.groupby('place_name').price_per_m2.transform(lambda x: x.fillna(x.mean()))
# precios_caba.isnull().sum() 

#Parece que funciono!!!!

### 15. Describe() de los grupos nuevamente para ver como quedaron los datos estadisticos

### 16. Analisis del campo **description** para sacar cantidad de ambientes y amenities (si es posible) ###

In [ ]:
description_series = data_caba_barrios['description']
description_series.sample(10)


#### Sacamos cantidad de ambientes a traves de expresiones regulares ####

In [ ]:
amb1_pattern = "((?P<numero_amb>\d)(\s)?(amb|AMB))"
amb1_pattern_regex = re.compile(amb1_pattern)

In [ ]:
amb1_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      amb1_pattern_regex.search(x))
mask_amb1_match_notnull = amb1_match.notnull()

data_caba_barrios.loc[mask_amb1_match_notnull, 'number_rooms'] = \
amb1_match[mask_amb1_match_notnull].apply(lambda x: x.group("numero_amb"))

In [ ]:
data_caba_barrios['number_rooms'].sample(15)

In [ ]:
data_caba_barrios['number_rooms'].dtypes
data_caba_barrios['number_rooms'].notnull().sum()

In [ ]:
dorm1_pattern = "((?P<numero_dorm>\d)(\s)?(dorm|DORM))"
dorm1_pattern_regex = re.compile(dorm1_pattern)

In [ ]:
dorm1_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else dorm1_pattern_regex.search(x))

mask_dorm1_match_notnull = dorm1_match.notnull()

data_caba_barrios.loc[mask_dorm1_match_notnull, 'number_rooms'] = dorm1_match[mask_dorm1_match_notnull].apply(lambda x: x.group("numero_dorm"))
#se asume que el numero de ambientes es numero_dorm +1 FALTA SUMAR UNO A ESE NUMERO (ver ejemplo de century en Clase 5)

In [ ]:
data_caba_barrios['number_rooms'].head(15)


In [ ]:
data_caba_barrios['number_rooms'].notnull().sum()

In [ ]:
#Falta Ambiente unico, mono(\s)?ambiente, 1 y 1/2 amb / dos/tres/cuatro/cinco ([a-zA-Z]*)? dormi

#PISOS: \d piso, planta baja, PH = planta baja, \der\spiso, con palabras?

#Amemities (pileta/piscina, SUM, laundry/lavadero, cochera/garage, Gimnasio/gym, seguridad)

### 17. Convertir en dummies los valores del campo descripcion y tipo de propiedad ###


### 18. Grafico de distribucion con geolocalizacion / cantidad de ventas por barrio segun tipo de propiedad ###